In [2]:
# Python Imports
import networkx as nx
import ipywidgets as widgets
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

# Project Imports
from src.mesh_analysis.readers.file_reader import FileReader
from src.mesh_analysis.tracers.waku_tracer import WakuTracer
from src.mesh_analysis.visualizer.waku_visualizer import add_elapsed_time
from src.mesh_analysis.visualizer.waku_visualizer import prepare_dropdowns
from src.mesh_analysis.visualizer.waku_visualizer import get_node_position
from src.mesh_analysis.visualizer.waku_visualizer import display_msg_trace

In [3]:
tracer = WakuTracer()
reader = FileReader("../../../test/log_test/", tracer)

df = reader.read()
df = add_elapsed_time(df)

In [4]:
df

sender_peer_id receiver_peer_id  \
msg_hash timestamp                                                          
0x1      2024-04-22 14:06:58.001000+00:00              A                B   
         2024-04-22 14:06:58.002000+00:00              B                D   
         2024-04-22 14:06:58.003000+00:00              A                C   
         2024-04-22 14:06:58.003000+00:00              D                E   
         2024-04-22 14:06:58.006000+00:00              A                F   
0x2      2024-04-22 14:06:59.001000+00:00              A                B   

                                           elapsed_time  
msg_hash timestamp                                       
0x1      2024-04-22 14:06:58.001000+00:00             0  
         2024-04-22 14:06:58.002000+00:00             1  
         2024-04-22 14:06:58.003000+00:00             2  
         2024-04-22 14:06:58.003000+00:00             2  
         2024-04-22 14:06:58.006000+00:00             5  
0x2      2024-04-22 14:06:59.001000+00:00             0

In [6]:
%matplotlib

msg_d, t_d = prepare_dropdowns(df)
g_pos_layout = get_node_position(df)
    
fig = plt.figure(figsize=(10,10))
ax_ = fig.add_subplot(1, 1, 1)

i_plot = widgets.interactive(display_msg_trace, x=msg_d, y=t_d, g_pos_layout=widgets.fixed(g_pos_layout),data_=widgets.fixed(df), ax_=widgets.fixed(ax_))
i_plot


Using matplotlib backend: QtAgg


interactive(children=(SelectMultiple(description='Msg hash', layout=Layout(height='100px', width='100%'), opti…